## Background

You will use the data contained in the train.csv file to train a model that will predict **dissolved inorganic carbon (DIC)** content in the water samples.

In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1_l2
from keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers.experimental.preprocessing import Normalization
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

2024-03-20 05:19:38.996892: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-20 05:19:39.116626: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-20 05:19:39.120687: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/R/4.2.2/lib/R/lib:/lib:/usr/local/lib:/usr/lib/x86_64-linux-gnu:/usr/lib/jvm/j

In [2]:
import os
import tensorflow as tf

# Set the environment variable to change the log level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 0 = default, 1 = no INFO, 2 = no INFO and WARNING, 3 = no INFO, WARNING, and ERROR

In [3]:
# Turn off scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Set seed
np.random.seed(123)

In [4]:
# Import data
train_df = pd.read_csv('data/train.csv')
train_df.columns = train_df.columns.str.lower().str.replace(' ', '_')

# Data exploration
print(train_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454 entries, 0 to 1453
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 1454 non-null   int64  
 1   lat_dec            1454 non-null   float64
 2   lon_dec            1454 non-null   float64
 3   no2um              1454 non-null   float64
 4   no3um              1454 non-null   float64
 5   nh3um              1454 non-null   float64
 6   r_temp             1454 non-null   float64
 7   r_depth            1454 non-null   int64  
 8   r_sal              1454 non-null   float64
 9   r_dynht            1454 non-null   float64
 10  r_nuts             1454 non-null   float64
 11  r_oxy_micromol.kg  1454 non-null   float64
 12  unnamed:_12        0 non-null      float64
 13  po4um              1454 non-null   float64
 14  sio3um             1454 non-null   float64
 15  ta1.x              1454 non-null   float64
 16  salinity1          1454 

In [5]:
# Check distribution of outcome variable
#sns.histplot(train_df['dic'], kde=False)
#plt.show()


## Preprocess

In [6]:
# Remove 'id' and 'unnamed:_12' columns (for reasons specified above)
train_df = train_df.drop(['id', 'unnamed:_12'], axis=1)

## Build & train model

In [ ]:
def create_model(optimizer='adam', learn_rate=0.001, momentum=0.0,
                 activation='relu', l1_reg=0.01, l2_reg=0.01,
                 dropout_rate=0.0, neurons_layer1=64, neurons_layer2=64):
    # Assuming train_df is available to compute the normalization layer statistics
    normalizer = Normalization(axis=-1)
    normalizer.adapt(train_df.drop('dic', axis=1).to_numpy())
    
    model = Sequential([
        normalizer,  # Normalization layer for features
        Dense(neurons_layer1, activation=activation,
              kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)),
        Dropout(dropout_rate),
        Dense(neurons_layer2, activation=activation,
              kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg)),
        Dropout(dropout_rate),
        Dense(1, kernel_regularizer=l1_l2(l1=l1_reg, l2=l2_reg))  # Output layer
    ])
    
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Normalize 'dic' using StandardScaler
scaler = StandardScaler()
train_df['dic_scaled'] = scaler.fit_transform(train_df[['dic']])

# Update model wrapper
model = KerasRegressor(model=create_model, verbose=0)

# Update param_grid with new hyperparameters
param_grid = {
    'model__momentum': [0.0, 0.8, 0.9, 0.99],
    'model__activation': ['relu', 'tanh', 'sigmoid', 'linear', 'elu', 'selu'],
    'model__l1_reg': [0.001, 0.01, 0.1],
    'model__l2_reg': [0.001, 0.01, 0.1],
    'model__dropout_rate': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
}

# Initiate GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=10)

# Train using the features and the normalized outcome
grid_result = grid.fit(train_df.drop(['dic', 'dic_scaled'], axis=1), train_df['dic_scaled'], epochs=100, batch_size=16)

# Later, to transform predictions back to the original scale:
# predictions_original_scale = scaler.inverse_transform(predictions_scaled)

/opt/python/3.7.13/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [ ]:
# Retrieve the index of the best model
best_index = grid_result.best_index_

# Extract the mean and standard deviation of the MSE for the best model
mse = grid_result.cv_results_['mean_test_score'][best_index]
std_mse = grid_result.cv_results_['std_test_score'][best_index]

# Print the results
print(f"MSE for the best model during CV: {mse}")
print(f"Standard deviation of MSE for the best model during CV: {std_mse}")

## Predict testing data

In [ ]:
# Import data
test_df = pd.read_csv('data/test.csv')
test_df.columns = train_df.columns.str.lower().str.replace(' ', '_')

# define feature matrix
X_test = test_df.drop('dic', axis=1).values

# define target vector
y_test = test_df['dic'].values

In [ ]:

# Step 1: Retrieve the best model from grid search
best_model = grid_result.best_estimator_

# Step 2: Prepare test dataset (Make sure it's prepared in the same way as your training data)
# If you've applied any transformations to your training dataset, apply the same here
X_test = test_df.drop('dic', axis=1, errors='ignore')  # Assuming 'dic' might not be in your test dataset

# Step 3: Use the best model to make predictions on the test dataset
predictions = history.predict(X_test)

In [ ]:
# Import data
submission_df = pd.read_csv('data/sample_submission.csv')
submission_df.columns = submission_df.columns.str.lower().str.replace(' ', '_')

# Assuming 'predictions' is your vector of predicted values
submission_df['dic'] = predictions

submission_df

In [ ]:
submission_df.to_csv('linus_submission5.csv', index=False)